In [1]:
import  os
os.chdir('../')

In [9]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir : Path
    data_path : Path
    model_path : Path
    all_params: dict
    tokenizer_path : Path
    metric_file_name : Path

In [10]:
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml, create_directories, save_json, load_json

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_name = config.metric_file_name,
            all_params = params
           
        )

        return model_evaluation_config

In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk, load_metric
import torch
import pandas as pd
from tqdm import tqdm

[2024-08-11 20:23:00,587: INFO: config: PyTorch version 2.2.2+cu121 available.]
[2024-08-11 20:23:00,589: INFO: config: TensorFlow version 2.12.0 available.]


In [11]:
import mlflow
import dagshub
import json

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer, 
                               batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu", 
                               column_text="article", 
                               column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):
            
            inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                            padding="max_length", return_tensors="pt")
            
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device), 
                            length_penalty=0.8, num_beams=8, max_length=128)
            
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                    clean_up_tokenization_spaces=True) 
                for s in summaries]      
            
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
            
            metric.add_batch(predictions=decoded_summaries, references=target_batch)
            
        score = metric.compute()
        return score

    def evaluate(self):
        # Set up MLflow tracking
        dagshub.init(repo_owner='azizulhakim8291', repo_name='text-summarization', mlflow=True)
        mlflow.set_tracking_uri("https://dagshub.com/azizulhakim8291/text-summarization.mlflow")
        mlflow.set_experiment("text-summarization-evaluation")

        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
       
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
        rouge_metric = load_metric('rouge')

        with mlflow.start_run():
            mlflow.log_param("model_name", "pegasus")
            mlflow.log_param("dataset", "samsum")
            mlflow.log_param('parameter name', 'value')

            score = self.calculate_metric_on_test_ds(
                dataset_samsum_pt['test'][0:10], rouge_metric, model_pegasus, tokenizer, 
                batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
            )

            rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
            mlflow.log_params(self.config.all_params)

            # Log metrics to MLflow
            for rouge_name, rouge_score in rouge_dict.items():
                mlflow.log_metric(rouge_name, rouge_score)

            # Save results as JSON
            with open(self.config.metric_file_name, 'w') as f:
                json.dump(rouge_dict, f, indent=4)

            # Log the JSON file as an artifact
            mlflow.log_artifact(self.config.metric_file_name)

In [12]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise e

[2024-08-11 22:39:28,983: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-11 22:39:28,986: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-11 22:39:28,989: INFO: common: created directory at: artifacts]
[2024-08-11 22:39:28,992: INFO: common: created directory at: artifacts/model_evaluation]
[2024-08-11 22:39:29,723: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/azizulhakim8291/text-summarization "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "azizulhakim8291/text-summarization"

[2024-08-11 22:39:29,731: INFO: helpers: Initialized MLflow to track repo "azizulhakim8291/text-summarization"]


Repository azizulhakim8291/text-summarization initialized!

[2024-08-11 22:39:29,735: INFO: helpers: Repository azizulhakim8291/text-summarization initialized!]
[2024-08-11 22:39:29,802: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /azizulhakim8291/text-summarization.mlflow/api/2.0/mlflow/experiments/get-by-name?experiment_name=text-summarization-evaluation]


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
100%|██████████| 5/5 [00:17<00:00,  3.48s/it]

[2024-08-11 22:39:59,553: INFO: rouge_scorer: Using default tokenizer.]
